[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EMQA_elec_seasonality/EMQA_elec_seasonality.ipynb)

# EMQA_elec_seasonality

Multiple seasonalities in electricity prices.

**Output:** `electricity_seasonality.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import os
def load_elec():
    paths = [
        '../../charts/electricity_cache.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/electricity_cache.csv',
    ]
    for p in paths:
        if os.path.exists(p):
            df = pd.read_csv(p, parse_dates=[0], index_col=0)
            return df
    # Fallback: generate from German data
    paths2 = [
        '../../charts/german_dayahead_prices.csv',
        '/Users/danielpele/Documents/Energy MBA/charts/german_dayahead_prices.csv',
    ]
    for p in paths2:
        if os.path.exists(p):
            return pd.read_csv(p, parse_dates=[0], index_col=0)
    raise FileNotFoundError("No electricity data found")

elec = load_elec()
# Get price column
if 'price' in elec.columns:
    price_col = 'price'
elif 'Price' in elec.columns:
    price_col = 'Price'
else:
    price_col = elec.columns[0]
elec_price = elec[price_col].dropna()


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

hourly = elec_price.groupby(elec_price.index.hour).mean()
axes[0].bar(range(24), hourly.values, color=COLORS['blue'], alpha=0.7)
axes[0].set_title('Intraday Pattern', fontweight='bold')
axes[0].set_xlabel('Hour'); axes[0].set_ylabel('EUR/MWh')

daily = elec_price.groupby(elec_price.index.dayofweek).mean()
axes[1].bar(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], daily.values,
            color=[COLORS['blue']]*5+[COLORS['orange']]*2, alpha=0.7)
axes[1].set_title('Weekly Pattern', fontweight='bold')
axes[1].set_ylabel('EUR/MWh')

monthly = elec_price.groupby(elec_price.index.month).mean()
months = ['J','F','M','A','M','J','J','A','S','O','N','D']
axes[2].bar(range(12), monthly.values, color=COLORS['green'], alpha=0.7)
axes[2].set_xticks(range(12)); axes[2].set_xticklabels(months)
axes[2].set_title('Annual Pattern', fontweight='bold')
axes[2].set_ylabel('EUR/MWh')

fig.suptitle('Electricity: Multiple Seasonalities', fontweight='bold', fontsize=13)
fig.tight_layout(rect=[0,0,1,0.95])
save_fig(fig, 'electricity_seasonality.pdf')
plt.show()
